In [1]:
'''
This file is for the index setup by Whoosh.
It takes the wiki-pages-text.zip as the input file
'''

import os,zipfile,time
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
import sys

def createSearchableData(zippath):   

    schema = Schema(title=TEXT(stored=True),content=TEXT(stored=True),title_remove_underline=TEXT(stored=True))
    if not os.path.exists("indexdir"):
        os.mkdir("indexdir")
 
    # Creating a index writer to add document as per schema
    ix = create_in("indexdir",schema)
    writer = ix.writer()
    
    wiki = zipfile.ZipFile(zippath,'r')
    
    for wikipage in wiki.namelist():
        if wikipage != 'wiki-pages-text/':
            with wiki.open(wikipage,'r') as page:

                firstline = page.readline().decode('utf-8')
                wordlist = firstline.split()
                current_title = wordlist[0]
                current_content = firstline

                for line in page:
                    sentence = line.decode('utf-8')
                    wordlist = sentence.split()
                    wikititle = wordlist[0]
                    number = wordlist[1]
                    if not str.isdigit(number):
                        continue
                    if wikititle == current_title:
                        current_content += sentence
                    else:
                        current_title_by_word = current_title.replace("_", " ")
                        writer.add_document(title=current_title, content=current_content, title_remove_underline=current_title_by_word)
                        current_title = wikititle
                        current_content = sentence
                
    writer.commit()
    wiki.close()
    

In [ ]:
time_start=time.time()
createSearchableData('wiki-pages-text.zip')
time_end=time.time()
print('time cost',time_end-time_start,'s')